In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import pysrt

In [13]:
df = pd.read_csv(r"big.csv",encoding='latin1')

In [14]:
df.groupby('gender').describe()

FileName                                                   Content  \
          count unique                                   top freq   count   
gender                                                                      
0          4017   4017  000235a2ba2f48231b7d24e1f08d7878.txt    1    4017   
1          5483   5483  0000e06e07496624211632e8e264126c.txt    1    5483   

                                                                       
       unique                                                top freq  
gender                                                                 
0        4017  damn these 2 games kicked my ass today.,huge w...    1  
1        5483  Paranoia is settin'in!,\Sumbuddy's gonna get a...    1

In [15]:
df.shape


(9500, 3)

In [16]:
df['gender'].value_counts()

1    5483
0    4017
Name: gender, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Content'],df['gender'], stratify=df['gender'])

In [8]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


2023-01-27 23:59:42.106254: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-27 23:59:42.106291: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-27 23:59:42.106305: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (455team3): /proc/driver/nvidia/version does not exist
2023-01-27 23:59:42.106465: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])


In [10]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [11]:
len(X_train)


7125

In [12]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)


In [13]:
model.fit(X_train, y_train, epochs=8)


Epoch 1/10
223/223 [==============================] - 1634s 7s/step - loss: 0.6790 - accuracy: 0.5830
Epoch 2/10
223/223 [==============================] - 1369s 6s/step - loss: 0.6505 - accuracy: 0.6125
Epoch 3/10
223/223 [==============================] - 1130s 5s/step - loss: 0.6366 - accuracy: 0.6392
Epoch 4/10
223/223 [==============================] - 1134s 5s/step - loss: 0.6241 - accuracy: 0.6480
Epoch 5/10
223/223 [==============================] - 1127s 5s/step - loss: 0.6147 - accuracy: 0.6559
Epoch 6/10
223/223 [==============================] - 1128s 5s/step - loss: 0.6105 - accuracy: 0.6623
Epoch 7/10
223/223 [==============================] - 1006s 5s/step - loss: 0.6076 - accuracy: 0.6672
Epoch 8/10
223/223 [==============================] - 651s 3s/step - loss: 0.6047 - accuracy: 0.6681
Epoch 9/10
223/223 [==============================] - 652s 3s/step - loss: 0.6004 - accuracy: 0.6672
Epoch 10/10
223/223 [==============================] - 650s 3s/step - loss: 0.5945 -

In [14]:
from sklearn.metrics import classification_report
import numpy as np

y_predicted = model.predict(X_test).flatten()

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

print(classification_report(y_test, y_predicted))


              precision    recall  f1-score   support

           0       0.75      0.28      0.41      1004
           1       0.64      0.93      0.76      1371

    accuracy                           0.66      2375
   macro avg       0.69      0.61      0.58      2375
weighted avg       0.68      0.66      0.61      2375



In [15]:
model.save("~/text_classifier_v1_gender")

2023-01-28 02:59:17.345972: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ~/text_classifier_v1_gender/assets


INFO:tensorflow:Assets written to: ~/text_classifier_v1_gender/assets


In [16]:
import pickle

filename = 'finalized_model_gender.sav'
pickle.dump(model, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://38f5cb04-6f9d-4a11-8e17-16bc212eb125/assets


INFO:tensorflow:Assets written to: ram://38f5cb04-6f9d-4a11-8e17-16bc212eb125/assets
